<a href="https://colab.research.google.com/github/sulaimanbehzad/Clustering_according_to_item_popularity/blob/main/clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clustering of Digikala dataset based on items popularity  


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, normalize

### Cloning the github repository in colab

In [20]:
% cd /content/drive/MyDrive/Clustering_according_to_item_popularity/


/content/drive/MyDrive/Clustering_according_to_item_popularity


In [ ]:
! ls

clustring.ipynb  LICENSE  README.md


In [ ]:
! git clone https://github.com/sulaimanbehzad/Clustering_according_to_item_popularity.git

Cloning into 'Clustering_according_to_item_popularity'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 8 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (8/8), done.


In [24]:
! git status

On branch main
Your branch is up to date with 'origin/main'.

Changes to be committed:
  (use "git reset HEAD <file>..." to unstage)

	deleted:    clustring.ipynb



In [22]:
! git pull

remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 12 (delta 7), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (12/12), done.
From https://github.com/sulaimanbehzad/Clustering_according_to_item_popularity
   1f2eeea..6854a3e  main       -> origin/main
Updating 1f2eeea..6854a3e
Fast-forward
 clustering.ipynb | 666 +++++++++++++++++++++++++++++++++++++++++++++++++++++++
 clustring.ipynb  | 488 ++++++++++++++++++++++++++++++++++++++--
 2 files changed, 1135 insertions(+), 19 deletions(-)
 create mode 100644 clustering.ipynb


In [39]:
! git push -u origin main

fatal: could not read Username for 'https://github.com': No such device or address


In [28]:
! git commit -m 'redundant files removed'

[main 8780da0] redundant files removed
 1 file changed, 666 deletions(-)
 delete mode 100644 clustring.ipynb


### Reading and preprocessing the dataset

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/Clustering_according_to_item_popularity/keifiat.xlsx')

In [ ]:
df.head()

,product_id,product_title,title_en,user_id,likes,dislikes,verification_status,recommend,title,comment,advantages,disadvantages
0,3692,ماوس بی‌سیم لاجیتک مدل M325,IT,989472,0,0,verified,\N,NaN,واقعا عالیه. من که ازش خیلی راضیم,NaN,NaN
1,90213,شارژر همراه شیاومی مدل NDY-02-AN با ظرفیت 1000...,AC,3862150,4,1,verified,recommended,واقعاً عالیه,سلام، قبل اینکه نظرم رو بگم میخواستم به یک موض...,"[""عمر طولانی\r"",""افت بسیار کم میزان شارژ\r"",""ا...","[""ندارد""]"
2,59473,یدک پولیشر میکروفایبر مهسان مدل 20119,HW,626843,1,0,verified,not_recommended,خیلی سخت حوله اش در میاد,گیره های فلزی خیلی سخت تا میشوند و لذا حوله را...,NaN,NaN
3,120499,گوشی موبایل هوآوی آنر مدل 5X KIW-L21 دو سیم‌کارت,MO,786887,6,11,verified,no_idea,گوشی مقرون به صرفه,همه چیز در رابطه با ظاهر این گوشی بسیار خوب اس...,"[""صفحه نمایش پرنور و با کیفیت\r"",""کیفیت بالای ...","[""کیفیت پایین اسپیکر\r"",""حاشیه خالی زیر صفحه ن..."
4,67200,شارژر همراه شیائومی مدل Mi ظرفیت 5000 میلی آمپ...,AC,854531,19,4,verified,no_idea,ابعاد، استحکام و نگهداری شارژ مناسب,اگر ظرفیتش براتون کافیه حتما بخرید.\r\nیه شارژ...,"[""ظریف و زیبا و باریک\r"",""بدنه محکم و با دوام""]","[""ظرفیت متوسط""]"


In [ ]:
df.describe()

,product_id,user_id,likes,dislikes
count,100000.000000,1.000000e+05,100000.000000,100000.000000
mean,426482.510140,4.642959e+06,3.114790,1.649460
std,232470.511159,2.619034e+06,8.050031,8.912705
min,300.000000,4.661320e+05,0.000000,0.000000
25%,220871.250000,2.613059e+06,0.000000,0.000000
50%,427858.000000,4.710290e+06,1.000000,0.000000
75%,619397.250000,7.094806e+06,4.000000,1.000000
max,909423.000000,8.623288e+06,854.000000,1344.000000


Select the attributes that will give information about the popularity of the item

In [ ]:
df1 = df[['product_id', 'likes', 'dislikes','comment', 'recommend']]

Check the dataframe for null values and handle them

In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   product_id  100000 non-null  int64 
 1   likes       100000 non-null  int64 
 2   dislikes    100000 non-null  int64 
 3   comment     99883 non-null   object
 4   recommend   100000 non-null  object
dtypes: int64(3), object(2)
memory usage: 3.8+ MB


In [ ]:
df1.isnull().sum()

product_id      0
likes           0
dislikes        0
comment       117
recommend       0
dtype: int64

looking at the database 'recommend' we can see that \N values also represent null values which wouldn't be considered in the above

In [ ]:
df1['recommend'].value_counts()

recommended        36972
\N                 36382
not_recommended    16110
no_idea            10536
Name: recommend, dtype: int64

we can see that beside the 117 missing values we have 36382 records with \N values  
now we understand tha \N too should be considered as missing value


In [41]:
missing_values = ["\\N", "na", "--", "NA", "n/a"]
df = pd.read_excel('/content/drive/MyDrive/Clustering_according_to_item_popularity/keifiat.xlsx', na_values = missing_values)
df1 = df[['product_id', 'likes', 'dislikes','comment', 'recommend']]


after importing database with newly found aspects on NA we are going to handle null values

In [54]:
df1.isnull().sum()

product_id        0
likes             0
dislikes          0
comment         118
recommend     36382
dtype: int64

In [53]:
df1['recommend'].value_counts()

recommended        36972
not_recommended    16110
no_idea            10536
Name: recommend, dtype: int64

In [48]:
most_common = df1["recommend"].value_counts().idxmax
# df1['recommend'].fillna(median, inplace=True)
most_common

<bound method Series.idxmax of recommended        36972
not_recommended    16110
no_idea            10536
Name: recommend, dtype: int64>

replacing the null values with one of the above would falsely impact the clustering so let's take another approach:  
1. use group by to obtain groups of products with the same product_id
2. assign the group most_common value to the null ones

In [58]:
 groups = pd.concat(g for _, g in df1.groupby("product_id") if len(g) > 1)
 groups

,product_id,likes,dislikes,comment,recommend
35476,300,1,0,راستش رو بخواین وقتی رفتم کارتریجش زرو شارژ کن...,NaN
36899,300,0,1,برای کارهای اداری عالیه و به درد اونایی که زیا...,NaN
53425,300,1,0,حرف نداره برای کارای سنگین محشره,NaN
68494,300,1,0,بهترین پرینتری که تا به حال استفاده کردم\r\nکم...,NaN
76639,300,1,0,فوق العادس\r\nتنها ایرادش نداشتن wifi هست,NaN
...,...,...,...,...,...
97319,903494,1,1,واقعا نیم بوت محکمی هست و از کیفیت خیلی خوبی ب...,NaN
97347,903494,1,1,از لحاظ کیفیت ساخت هم خیلی خوبه و هم با دوام,NaN
97294,903513,3,1,خیلی راحته و جنس خیلی خوبی داره,NaN
97308,903513,3,2,در یک کلام بسیار بوت قشنگی هست و خیلی کیفیت خو...,NaN


In [69]:
for item in groups:
  print(str(item) + str(type(item)))



product_id<class 'str'>
likes<class 'str'>
dislikes<class 'str'>
comment<class 'str'>
recommend<class 'str'>
